In [1]:
!pip install qiskit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from qiskit import *

In [3]:
def is_rectangle (A : int,B : int,C : int,D : int):

  #Finding max length of binary string needed
  ss=[]
  ss.append(len(bin(A)[2:]))
  ss.append(len(bin(B)[2:]))
  ss.append(len(bin(C)[2:]))
  ss.append(len(bin(D)[2:]))
  n=max(ss) #n acts as register size for each number

  #converting all integers into binary strings of max size
  s=[]
  s.append((bin(A)[2:]).zfill(n))
  s.append((bin(B)[2:]).zfill(n))
  s.append((bin(C)[2:]).zfill(n))
  s.append((bin(D)[2:]).zfill(n))

  #Function to apply X gate according to string for each of the 4 registers
  def build_register(n,s) :
    qcb=QuantumCircuit(n)
    for i in range(n) :
      if s[i]=='1' :
        qcb.x(n-1-i)
    return qcb

  #This function evaluates the results of all side comparisions a-b,a-c,a-d,b-c,b-d,c-d
  def solution_reg():
    qcs=QuantumCircuit(9)
    for i in range(3,6) :
      qcs.ccx(i,i+3,2)
    qcs.cx(5,1)
    qcs.cx(4,1)
    qcs.cx(3,1)
    qcs.ccx(2,1,0)
    return qcs

  qc=QuantumCircuit(4*n+9,4*n+9)
  for i in range(4) :
    qc.compose(build_register(n,s[i]) ,qubits=range(9+i*n,9+n+i*n),inplace=True)
  
  #This gate compares two sides
  compare=QuantumCircuit(2*n+1)
  compare.cx(list(range(0,n)),list(range(n,2*n)))
  compare.x(range(n,2*n))
  compare.mcx(list(range(n,2*n)),2*n)
  compare.x(range(n,2*n))
  my_gate=compare.to_gate()
  #list of all the comparisions to be done
  k=[[0,1],[0,2],[0,3],[3,2],[1,3],[3,2]]
  for i in range(6) :
    lis=list(range(9+k[i][0]*n,9+k[i][0]*n+n))+list(range(9+k[i][1]*n,9+k[i][1]*n+n))
    lis.append(3+i)
    qc.append(my_gate,lis)
  
  qc.compose(solution_reg() ,qubits=range(0,9),inplace=True)
  qc.measure(range(9+4*n),range(9+4*n))
  backend = Aer.get_backend('qasm_simulator')
  result = execute(qc, backend).result()
  counts = result.get_counts()
  output = list(counts.keys())[0]
   
  #only Qubit 0 has the output and thus it would at end of string 
  return output[4*n+8]


In [4]:
A = is_rectangle(5,6,6,5)
print(A)

1


Qubits are as Follows
*   qubits=0 to 8 are used to evaluate comparisions
*   qubits 9 to 9+4*n-1 are used to represent 4 sides in qubits
<hr>

How does comparisions happen ? \\
Let sides be A, B, C and D \\
Comparisions are done using cx gates here (acts as XOR operator) \\
A and B are equal if A XOR B =0 and converse is true as well \\

IF B is zero after cx(A,B) then mcx gate would flip the register storing the comparision into corresponding qubit \\
0 if unequal and 1 if equal 

1.   A XOR B in qubit 3
2.   A XOR C in qubit 4
3.   A XOR D in qubit 5
4.   C XOR D in qubit 6
5.   B XOR D in qubit 7
6.   B XOR C in qubit 8
<hr>

Now a quadrilateral is a rectangle 
1. If two pairs of sides are equal
2. All sides are equal

Now favourable cases in q3 to q8 are 
1.   100100
2.   010010
3.   001001
4.   111111


rest all cases either are invalid or not a rectangle
<hr>

We can observe that first three qubits are equal to last 3 qubits and thus comparing those should be 1 incase of equal and 0 if not equal \\

We do this and store in qubit 2
But this still leaves us with cases 000000,110110,101101,011110 which doesnt form a rectangle \\
But No of 1s in half first three must always be odd. So we check if its odd or not in Qubit 1
<hr>

If both cases are valid then It must be rectangle and as such we use a ccx operator from qubits 2,1 as controls and 0 as target this if both are valid Qubit zero would be 1 and sides can form a rectangle